In [1]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.base import Pack, Simulation
from flowermd.library import OPLS_AA_PPS,PPS
from flowermd.utils import get_target_box_mass_density
import unyt as u
import hoomd

cpu = hoomd.device.CPU()

In [4]:
pps_chains = PPS(lengths=[15,17,19,22,30], num_mols=[3,2,1,3,1])
system = Pack(molecules=[pps_chains], density=0.1)
system.apply_forcefield(
    r_cut=2.5,
    force_field=OPLS_AA_PPS(),
    auto_scale=True,
    remove_hydrogens=True)

In [ ]:
gsd_path=('cg-pps-trajectory0.gsd')
log_path=('cg-pps-log0.txt')
sim = Simulation.from_system(
    system=system,
    gsd_write_freq=int(1e5),
    gsd_file_name=gsd_path,
    log_file_name=log_path,
    dt=0.001,
    log_write_freq=int(1e5),
)
#target_box = get_target_box_mass_density(density=system.density, mass=system.mass.to("g"))
#lj_sim.run_update_volume(final_box_lengths=target_box,kT=5.0,n_steps=1e4,tau_kt=5*lj_sim.dt,period=10)
sim.run_NVT(n_steps=int(1e7), kT=5.0, tau_kt=10*sim.dt)
sim.flush_writers()
sim.save_restart_gsd("pps_restart.gsd")

*Warning*: charge.pppm: system is not neutral and unscreened interactions are calculated, the net charge is -1797.76
*Warning*: charge.pppm: RMS error of 0.20868 is probably too high! 0.20868 0.20862


Initializing simulation state from a gsd.hoomd.Frame.


In [ ]:
from cmeutils.sampling import is_equilibrated
shrink_cut = 0
log = np.genfromtxt('log1.txt', names=True)
pe = log["mdcomputeThermodynamicQuantitiespotential_energy"]
# equil: bool (True or False), t0: int (starting data point that you can sample from), neff: number of data points in equilibrated region
print(pe[shrink_cut:])
equil, t0, g, neff = is_equilibrated(data=pe[shrink_cut:], threshold_neff=100) 
if equil:
    print("True")
else:
    # If it's not equilibrated, job.doc.equilibrated stays false, and the "run-longer" operation in project.py will run next time we submit this job
    print("False")